In [1]:
import os
import sys
sys.path.append(os.pardir)
import numpy as np
from numpy.random import *
from scipy import ceil, complex64, float64, hamming, zeros
from matplotlib import pylab as plt
import xml.etree.ElementTree as et
import argparse
import time
from cmf.cnimf import CNIMF
from matplotlib import pylab as plt
%matplotlib inline

In [2]:
convolution_width = 3
true_n_components = 2

In [3]:
parser = argparse.ArgumentParser(description='structure detection test')
parser.add_argument('-s', '--seed_number', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default=0, \
                    type=int, \
                    choices=None, \
                    help='seed_number', \
                    metavar=None)
parser.add_argument('-d', '--dat_dir', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default='../dat/structure_detection', \
                    type=str, \
                    choices=None, \
                    help='Directory where npy files will be stored.', \
                    metavar=None)
parser.add_argument('-n', '--dat_npz', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default='structure_result_{}_{}_{}.npz'.format(convolution_width, true_n_components, time.strftime('%Y%m%d%H%M')), \
                    type=str, \
                    choices=None, \
                    help='npz file name', \
                    metavar=None)


_StoreAction(option_strings=['-n', '--dat_npz'], dest='dat_npz', nargs='?', const=None, default='structure_result_3_2_201705131527.npz', type=<class 'str'>, choices=None, help='npz file name', metavar=None)

In [4]:
args = parser.parse_args([])

In [5]:
seed(args.seed_number)


In [6]:
n_tests = 50
n_criteria = 5
n_samples_list = [100, 200, 300, 400, 500, 600, 700, 800]
missing_rate_list = [0.2, 0.6]
data_dim = 12
convolution_width = convolution_width
true_n_components_list = [true_n_components]
gamma_shape = 2.0
gamma_rate = 2.0
base_max = 10.0
best_n_bases = np.zeros([len(true_n_components_list), len(n_samples_list), n_tests, n_criteria])
completion_error = np.zeros([len(true_n_components_list), len(missing_rate_list), len(n_samples_list), n_tests, n_criteria])
accuracy = np.zeros([len(true_n_components_list), len(n_samples_list), n_criteria])

In [7]:
# %pdb
# import warnings
# warnings.filterwarnings('error')
for i_n_components in range(len(true_n_components_list)):
    true_n_components = true_n_components_list[i_n_components]
    for i_n_samples in range(len(n_samples_list)):
        n_samples = n_samples_list[i_n_samples]
        print('n_samples', n_samples)
        for i_test in range(n_tests):
            print('i_test', i_test)
            true_activation = np.random.gamma(gamma_shape, 1.0 / gamma_rate, [n_samples, true_n_components])
            true_base = np.random.uniform(0.0, base_max, [convolution_width, true_n_components, data_dim])
            X = np.random.poisson(CNIMF.convolute(true_activation, true_base))
            arg_dict = dict(
                convolution_max = convolution_width,
                component_max = X.shape[1],
                true_width = convolution_width,
                gamma_shape = gamma_shape,
                gamma_rate = gamma_rate,
                base_max = base_max,
                convergence_threshold = 0.00001,
                loop_max = 1000)
            factorizer = CNIMF(**arg_dict)
            filtre = np.ones(X.shape)
            factorizer.fit(X, None, filtre)
            for i_criterion in range(n_criteria):
                best_n_bases[i_n_components, i_n_samples, i_test, i_criterion] = factorizer.criteria[i_criterion].best_structure[1]
            for i_missing_rate in range(len(missing_rate_list)):
                missing_rate = missing_rate_list[i_missing_rate]
                print(missing_rate)
                factorizer = CNIMF(**arg_dict)
                filtre = np.random.binomial(1, missing_rate, X.shape)
                factorizer.fit(X, None, filtre)
                for i_criterion in range(n_criteria):
                    completion_error[i_n_components, i_missing_rate, i_n_samples, i_test, i_criterion] = factorizer.criteria[i_criterion].completion_error/np.prod(X.shape)
                print(completion_error[i_n_components, i_missing_rate, i_n_samples, i_test, :])
    accuracy[i_n_components, :, :] = np.mean(best_n_bases[i_n_components, :, :, :]==true_n_components, axis = 1)

n_samples 100
i_test 0
0.2
[ 1.71882937  1.71882937  1.71882937  1.71882937  1.71882937]
0.6
[ 0.6981427   0.51230814  0.64975641  0.51230814  0.51230814]
i_test 1
0.2
[ 1.77651884  1.42417422  1.42417422  1.42417422  1.42417422]
0.6
[ 0.62694278  0.5551848   0.5551848   0.5551848   0.5551848 ]
i_test 2
0.2
[ 2.33712094  2.20022336  2.33712094  2.20022336  2.20022336]
0.6
[ 0.568357    0.5941583   0.55530333  0.5941583   0.5941583 ]
i_test 3
0.2
[ 2.26144423  2.17772625  2.17772625  2.17772625  2.17772625]
0.6
[ 0.72289488  0.53617252  0.53617252  0.53617252  0.53617252]
i_test 4
0.2
[ 2.2334023   1.83079421  1.83079421  1.83079421  1.83079421]
0.6
[ 0.7339273   0.67353504  0.67353504  0.67353504  0.61025466]
i_test 5
0.2
[ 2.02252048  1.96413435  1.96413435  1.96413435  1.96413435]
0.6
[ 0.6312436   0.47093131  0.47093131  0.47093131  0.47093131]
i_test 6
0.2
[ 2.03982938  1.92692495  1.92692495  1.92692495  1.92692495]
0.6
[ 0.60122347  0.53225478  0.53225478  0.53225478  0.53225478]

In [8]:
np.savez(args.dat_dir + '/' + args.dat_npz,
         accuracy=accuracy,
         completion_error=completion_error,
         best_n_bases=best_n_bases,
         n_samples_list = n_samples_list,
         missing_rate_list = missing_rate_list, 
         true_n_components_list = true_n_components_list,
         convolution_width = convolution_width)

In [9]:
# npz = np.load(args.dat_dir + '/artificial_result_201701210117.npz')

In [10]:
accuracy

array([[[ 0.  ,  0.7 ,  0.32,  0.64,  0.94],
        [ 0.  ,  0.  ,  0.04,  0.56,  0.88],
        [ 0.  ,  0.  ,  0.  ,  0.38,  0.78],
        [ 0.  ,  0.  ,  0.  ,  0.16,  0.84],
        [ 0.  ,  0.  ,  0.  ,  0.04,  0.72],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.54],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.3 ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.14]]])

In [11]:
completion_error

array([[[[[ 1.71882937,  1.71882937,  1.71882937,  1.71882937,  1.71882937],
          [ 1.77651884,  1.42417422,  1.42417422,  1.42417422,  1.42417422],
          [ 2.33712094,  2.20022336,  2.33712094,  2.20022336,  2.20022336],
          ..., 
          [ 1.96754328,  2.68094709,  2.68094709,  2.68094709,  2.68094709],
          [ 1.66130923,  1.34852655,  1.34852655,  1.34852655,  1.34852655],
          [ 1.81007744,  1.7178513 ,  1.7178513 ,  1.7178513 ,  1.7178513 ]],

         [[ 1.69231545,  1.69231545,  1.08961326,  1.08961326,  1.08961326],
          [ 1.80686821,  1.4656742 ,  1.4656742 ,  1.4656742 ,  1.4656742 ],
          [ 1.42227738,  1.26030613,  1.26030613,  0.88882171,  0.88882171],
          ..., 
          [ 1.79104982,  1.79104982,  1.79104982,  1.59937084,  1.59937084],
          [ 1.31205206,  1.4042398 ,  1.4042398 ,  1.01869997,  1.01869997],
          [ 1.96959048,  1.96959048,  1.96959048,  1.96959048,  1.48876487]],

         [[ 1.64157268,  1.52213894,  1.